In [3]:
import pandas as pd
import numpy as np

In [4]:
base_columns = ["Open", "High", "Low", "Close"]
index_columns = ["OpenSign", "HighSign", "LowSign", "CloseSign"]
preds_df = pd.read_csv("../outputs/predictions/sequifier-l3-d48-8targets-best-100-predictions.csv")
preds = {c: preds_df[c].values for c in (base_columns + index_columns)}


In [11]:
row_data = pd.read_parquet("../data/gspc-return-split2.parquet")
map= {1:"+", 2:"-"}
targets = {}
for c, group in row_data.groupby("inputCol"):
    if c in base_columns:
        targets[c] = group["target"].values
    if c in index_columns:
        targets[c] = np.array([map[v] for v in group["target"].values])


In [14]:
for c in base_columns:
    print(f"{c}: {np.corrcoef(preds[c], targets[c])[0, 1]}")
for c in index_columns:
    print(f"{c}: {np.mean(preds[c] == targets[c])}")

Open: -0.01151107824684262
High: 0.008608810021756882
Low: 0.013785708211568954
Close: 0.003551580367430096
OpenSign: 0.5068883610451307
HighSign: 0.4997624703087886
LowSign: 0.5130641330166271
CloseSign: 0.5068883610451307


In [19]:
dummy_preds = {} # data from prior day
for c, group in row_data.groupby("inputCol"):
    dummy_preds[c] = group["target"].values[:-1]
for c in base_columns:
    print(f"{c}: {np.corrcoef(dummy_preds[c], targets[c][1:])[0, 1]}")
for c in index_columns:
    print(f"{c}: {np.mean( np.array([map[v] for v in dummy_preds[c]]) == targets[c][1:])}")

Open: -0.07413460322069963
High: 0.11163872272367174
Low: 0.09875004819631697
Close: -0.12091067042500168
OpenSign: 0.4866920152091255
HighSign: 0.5674904942965779
LowSign: 0.5475285171102662
CloseSign: 0.46387832699619774
